Name: Perez, Cris John M.<br>
Course and Section: CPE 019 - CPE32S9<br>
Date of Submission: April 15, 2024<br>
Instructor: Engr. Roman Richard<br>

# Assignment 7.1: Classifications and Regression

#### Objective(s):

1.   Choose any dataset applicable to the classification problem, and also, choose any dataset applicable to the regression problem.
2.   Explain your datasets and the problem being addressed.
3. For classification, do the following:


*   Create a base model
*   Evaluate the model with k-fold cross validation
* Improve the accuracy of your model by applying additional hidden layers

4. For regression, do the following:


*   Create a base model
*   Improve the model by standardizing the dataset
* Show tuning of layers and neurons (see evaluating small and larger networks)





# Classification

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras.models  import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop

import seaborn as sns

%matplotlib inline

In [4]:
pip install ucimlrepo

In [5]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
raisin = fetch_ucirepo(id=850)

# data (as pandas dataframes)
X = raisin.data.features
y = raisin.data.targets

# metadata
print(raisin.metadata)

# variable information
print(raisin.variables)

{'uci_id': 850, 'name': 'Raisin', 'repository_url': 'https://archive.ics.uci.edu/dataset/850/raisin', 'data_url': 'https://archive.ics.uci.edu/static/public/850/data.csv', 'abstract': 'Images of the Kecimen and Besni raisin varieties were obtained with CVS. A total of 900 raisins were used, including 450 from both varieties, and 7 morphological features were extracted.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 900, 'num_features': 7, 'feature_types': ['Real', 'Integer'], 'demographics': [], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Fri Jan 05 2024', 'dataset_doi': '10.24432/C5660T', 'creators': ['İ̇lkay Çinar', 'Murat Koklu', 'Sakir Tasdemir'], 'intro_paper': {'title': 'Kuru Üzüm Tanelerinin Makine Görüşü ve Yapay Zeka Yöntemleri Kullanılarak Sınıflandırılması', 'authors': 'İ̇lkay Çinar, Murat Koklu, Sakir Tasdemir

This dataset classifies 2 raisin varieties: Kecimen and Besni. Images of 900 raisins were used, 450 from each variety.

### Cleaning and processing of the data

In [6]:
data = pd.concat([X, y], axis=1, join='inner')
data

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen
...,...,...,...,...,...,...,...,...
895,83248,430.077308,247.838695,0.817263,85839,0.668793,1129.072,Besni
896,87350,440.735698,259.293149,0.808629,90899,0.636476,1214.252,Besni
897,99657,431.706981,298.837323,0.721684,106264,0.741099,1292.828,Besni
898,93523,476.344094,254.176054,0.845739,97653,0.658798,1258.548,Besni


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             900 non-null    int64  
 1   MajorAxisLength  900 non-null    float64
 2   MinorAxisLength  900 non-null    float64
 3   Eccentricity     900 non-null    float64
 4   ConvexArea       900 non-null    int64  
 5   Extent           900 non-null    float64
 6   Perimeter        900 non-null    float64
 7   Class            900 non-null    object 
dtypes: float64(5), int64(2), object(1)
memory usage: 56.4+ KB


In [8]:
data["Class"] = data["Class"].apply(lambda toLabel: 0 if toLabel == 'Kecimen' else 1)
data

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,0
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,0
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,0
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,0
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,0
...,...,...,...,...,...,...,...,...
895,83248,430.077308,247.838695,0.817263,85839,0.668793,1129.072,1
896,87350,440.735698,259.293149,0.808629,90899,0.636476,1214.252,1
897,99657,431.706981,298.837323,0.721684,106264,0.741099,1292.828,1
898,93523,476.344094,254.176054,0.845739,97653,0.658798,1258.548,1


Splitting the train and test samples

In [9]:
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state = 1111, test_size = 0.25)

In [10]:
#Standardizing features
scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(x_train)
X_test_scaler = scaler.transform(x_test)

## Creating the base model

In [64]:
model_clf = Sequential()
model_clf.add(Dense(10, input_dim = X.shape[1], activation = 'relu'))
model_clf.add(Dense(8, activation='relu'))
model_clf.add(Dense(1, activation="sigmoid"))

In [65]:
model_clf.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_90 (Dense)            (None, 10)                80        
                                                                 
 dense_91 (Dense)            (None, 8)                 88        
                                                                 
 dense_92 (Dense)            (None, 1)                 9         
                                                                 
Total params: 177 (708.00 Byte)
Trainable params: 177 (708.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
model_clf.compile(optimizer=SGD(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
run_hist_1 = model_clf.fit(X_train_scaler, y_train, validation_data=(X_test_scaler, y_test), epochs=1000)

Epoch 1/1000
22/22 [==============================] - 1s 13ms/step - loss: 0.5785 - accuracy: 0.7289 - val_loss: 0.5568 - val_accuracy: 0.7511
Epoch 2/1000
22/22 [==============================] - 0s 4ms/step - loss: 0.5763 - accuracy: 0.7422 - val_loss: 0.5545 - val_accuracy: 0.7556
Epoch 3/1000
22/22 [==============================] - 0s 3ms/step - loss: 0.5742 - accuracy: 0.7467 - val_loss: 0.5523 - val_accuracy: 0.7467
Epoch 4/1000
22/22 [==============================] - 0s 4ms/step - loss: 0.5721 - accuracy: 0.7526 - val_loss: 0.5502 - val_accuracy: 0.7600
Epoch 5/1000
22/22 [==============================] - 0s 4ms/step - loss: 0.5702 - accuracy: 0.7600 - val_loss: 0.5483 - val_accuracy: 0.7689
Epoch 6/1000
22/22 [==============================] - 0s 4ms/step - loss: 0.5683 - accuracy: 0.7689 - val_loss: 0.5464 - val_accuracy: 0.7733
Epoch 7/1000
22/22 [==============================] - 0s 3ms/step - loss: 0.5665 - accuracy: 0.7748 - val_loss: 0.5446 - val_accuracy: 0.7733
Epoch

The accuracy of the model is 88.74% while the validation accuracy is 84%.



## Evaluation of the model using k-fold cross validation

In [67]:
import tensorflow
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping

In [68]:
k = 5
kf = KFold(n_splits=k, random_state=None, shuffle=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

fold_accuracies = []
train_loss = []
train_acc = []
val_loss = []
val_acc = []

for train_index, val_index in kf.split(X_train_scaler):
    X_fold_train, X_fold_val = X_train_scaler[train_index], X_train_scaler[val_index]
    y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

    run_hist_fold = model_clf.fit(X_fold_train, y_fold_train, validation_data=(X_fold_val, y_fold_val), epochs=1000, callbacks=[early_stopping])
    if early_stopping.stopped_epoch:
        print(f"Early stopping at epoch {early_stopping.stopped_epoch}")
    fold_accuracy = run_hist_fold.history['accuracy'][-1]
    train_loss.append(run_hist_fold.history['loss'][0])
    train_acc.append(run_hist_fold.history['accuracy'][0])
    val_loss.append(run_hist_fold.history['val_loss'][-1])
    val_acc.append(run_hist_fold.history['val_accuracy'][-1])
    fold_accuracies.append(fold_accuracy)


print("Mean fold accuracy: {}%".format(np.mean(fold_accuracies).round(3)))
# Print the mean and standard deviation of the loss and accuracy for each fold
print("Mean train loss: {}%".format(np.mean(train_loss).round(3)))
print("Mean train accuracy: {}%".format(np.mean(np.mean(train_acc)).round(3)))
print("Mean val loss: {}%".format(np.mean(np.mean(val_loss)).round(3)))
print("Mean val accuracy: {}%".format(np.mean(np.mean(val_acc)).round(3)))

Epoch 1/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.2995 - accuracy: 0.8944 - val_loss: 0.3406 - val_accuracy: 0.8667
Epoch 2/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.2994 - accuracy: 0.8944 - val_loss: 0.3406 - val_accuracy: 0.8667
Epoch 3/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.2994 - accuracy: 0.8944 - val_loss: 0.3407 - val_accuracy: 0.8667
Epoch 4/1000
17/17 [==============================] - 0s 4ms/step - loss: 0.2994 - accuracy: 0.8944 - val_loss: 0.3407 - val_accuracy: 0.8667
Epoch 5/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.2994 - accuracy: 0.8944 - val_loss: 0.3408 - val_accuracy: 0.8667
Epoch 6/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.2993 - accuracy: 0.8944 - val_loss: 0.3408 - val_accuracy: 0.8667
Epoch 7/1000
17/17 [==============================] - 0s 5ms/step - loss: 0.2993 - accuracy: 0.8944 - val_loss: 0.3408 - val_accuracy: 0.8667
Epoch 

## Improving the accuracy using hidden layers

In [69]:
model_clf_1 = Sequential()
model_clf_1.add(Dense(32, input_dim = X.shape[1], activation = 'relu'))
model_clf_1.add(Dense(32, activation='relu'))
model_clf_1.add(Dense(1, activation='sigmoid'))

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

model_clf_1.compile(optimizer=SGD(learning_rate=0.003), loss='binary_crossentropy', metrics=['accuracy'])

run_hist_2 = model_clf_1.fit(X_train_scaler, y_train, validation_data=(X_test_scaler, y_test), epochs=1500, callbacks=[early_stopping])

Epoch 1/1500
22/22 [==============================] - 1s 12ms/step - loss: 0.5510 - accuracy: 0.8504 - val_loss: 0.5213 - val_accuracy: 0.8444
Epoch 2/1500
22/22 [==============================] - 0s 4ms/step - loss: 0.5411 - accuracy: 0.8533 - val_loss: 0.5133 - val_accuracy: 0.8400
Epoch 3/1500
22/22 [==============================] - 0s 4ms/step - loss: 0.5314 - accuracy: 0.8652 - val_loss: 0.5053 - val_accuracy: 0.8356
Epoch 4/1500
22/22 [==============================] - 0s 4ms/step - loss: 0.5222 - accuracy: 0.8637 - val_loss: 0.4979 - val_accuracy: 0.8356
Epoch 5/1500
22/22 [==============================] - 0s 4ms/step - loss: 0.5136 - accuracy: 0.8667 - val_loss: 0.4906 - val_accuracy: 0.8311
Epoch 6/1500
22/22 [==============================] - 0s 4ms/step - loss: 0.5049 - accuracy: 0.8681 - val_loss: 0.4842 - val_accuracy: 0.8311
Epoch 7/1500
22/22 [==============================] - 0s 4ms/step - loss: 0.4974 - accuracy: 0.8652 - val_loss: 0.4781 - val_accuracy: 0.8311
Epoch

In [70]:
k = 5
kf = KFold(n_splits=k, random_state=None, shuffle=True)


fold_accuracies = []
train_loss = []
train_acc = []
val_loss = []
val_acc = []

for train_index, val_index in kf.split(X_train_scaler):
    X_fold_train, X_fold_val = X_train_scaler[train_index], X_train_scaler[val_index]
    y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

    run_hist_fold = model_clf_1.fit(X_fold_train, y_fold_train, validation_data=(X_fold_val, y_fold_val), epochs=1500, callbacks=[early_stopping])
    if early_stopping.stopped_epoch:
        print(f"Early stopping at epoch {early_stopping.stopped_epoch}")

    fold_accuracy = run_hist_fold.history['accuracy'][-1]
    train_loss.append(run_hist_fold.history['loss'][0])
    train_acc.append(run_hist_fold.history['accuracy'][0])
    val_loss.append(run_hist_fold.history['val_loss'][-1])
    val_acc.append(run_hist_fold.history['val_accuracy'][-1])
    fold_accuracies.append(fold_accuracy)


print("Mean fold accuracy: {}%".format(np.mean(fold_accuracies).round(3)))
# Print the mean and standard deviation of the loss and accuracy for each fold
print("Mean train loss: {}%".format(np.mean(train_loss).round(3)))
print("Mean train accuracy: {}%".format(np.mean(np.mean(train_acc)).round(3)))
print("Mean val loss: {}%".format(np.mean(np.mean(val_loss)).round(3)))
print("Mean val accuracy: {}%".format(np.mean(np.mean(val_acc)).round(3)))

Epoch 1/1500
17/17 [==============================] - 0s 14ms/step - loss: 0.3099 - accuracy: 0.9019 - val_loss: 0.4355 - val_accuracy: 0.8222
Epoch 2/1500
17/17 [==============================] - 0s 7ms/step - loss: 0.3095 - accuracy: 0.9019 - val_loss: 0.4357 - val_accuracy: 0.8222
Epoch 3/1500
17/17 [==============================] - 0s 8ms/step - loss: 0.3091 - accuracy: 0.9019 - val_loss: 0.4358 - val_accuracy: 0.8222
Epoch 4/1500
17/17 [==============================] - 0s 5ms/step - loss: 0.3087 - accuracy: 0.9019 - val_loss: 0.4360 - val_accuracy: 0.8222
Epoch 5/1500
17/17 [==============================] - 0s 5ms/step - loss: 0.3083 - accuracy: 0.9019 - val_loss: 0.4361 - val_accuracy: 0.8222
Epoch 6/1500
17/17 [==============================] - 0s 6ms/step - loss: 0.3079 - accuracy: 0.9037 - val_loss: 0.4363 - val_accuracy: 0.8222
Epoch 7/1500
17/17 [==============================] - 0s 6ms/step - loss: 0.3075 - accuracy: 0.9037 - val_loss: 0.4364 - val_accuracy: 0.8222
Epoch

Dataset used: Çinar,İ̇lkay, Koklu,Murat, and Tasdemir,Sakir. (2023). Raisin. UCI Machine Learning Repository. https://doi.org/10.24432/C5660T.

# Regression

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras.models  import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop

import seaborn as sns

%matplotlib inline

In [72]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
liver_disorders = fetch_ucirepo(id=60)

# data (as pandas dataframes)
X = liver_disorders.data.features
y = liver_disorders.data.targets

# metadata
print(liver_disorders.metadata)

# variable information
print(liver_disorders.variables)

{'uci_id': 60, 'name': 'Liver Disorders', 'repository_url': 'https://archive.ics.uci.edu/dataset/60/liver+disorders', 'data_url': 'https://archive.ics.uci.edu/static/public/60/data.csv', 'abstract': 'BUPA Medical Research Ltd. database donated by Richard S. Forsyth', 'area': 'Health and Medicine', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 345, 'num_features': 5, 'feature_types': ['Categorical', 'Integer', 'Real'], 'demographics': [], 'target_col': ['drinks'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2016, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C54G67', 'creators': [], 'intro_paper': None, 'additional_info': {'summary': 'The first 5 variables are all blood tests which are thought to be sensitive to liver disorders that might arise from excessive alcohol consumption. Each line in the dataset constitutes the record of a single male individual.\n\nImportant note: The 7

In [73]:
regdata = pd.concat([X, y], axis=1, join='inner')
regdata

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,85,92,45,27,31,0.0
1,85,64,59,32,23,0.0
2,86,54,33,16,54,0.0
3,91,78,34,24,36,0.0
4,87,70,12,28,10,0.0
...,...,...,...,...,...,...
340,99,75,26,24,41,12.0
341,96,69,53,43,203,12.0
342,98,77,55,35,89,15.0
343,91,68,27,26,14,16.0


This dataset contains data of blood tests which are thought to be sensitive to liver disorders that might arise from excessive alcohol consumption. It has 345 samples of which are all male.

In [74]:
regdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   mcv      345 non-null    int64  
 1   alkphos  345 non-null    int64  
 2   sgpt     345 non-null    int64  
 3   sgot     345 non-null    int64  
 4   gammagt  345 non-null    int64  
 5   drinks   345 non-null    float64
dtypes: float64(1), int64(5)
memory usage: 16.3 KB


Splitting train and test samples

In [75]:
x = regdata.iloc[:,:-1].values
y = regdata.iloc[:,-1].values

X_train, X_test, Y_train, Y_test = train_test_split(x, y, random_state = 1111, test_size = 0.25)

## Creation of base model

In [78]:
model_reg = Sequential()
model_reg.add(Dense(20, input_shape=(5,), activation='relu'))
model_reg.add(Dense(1))

model_reg.compile(optimizer='adam', loss='mse')
hist_3 = model_reg.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test))

Epoch 1/100
9/9 [==============================] - 1s 33ms/step - loss: 1421.7494 - val_loss: 923.1467
Epoch 2/100
9/9 [==============================] - 0s 8ms/step - loss: 707.2516 - val_loss: 412.6346
Epoch 3/100
9/9 [==============================] - 0s 5ms/step - loss: 318.0496 - val_loss: 169.6828
Epoch 4/100
9/9 [==============================] - 0s 5ms/step - loss: 155.4637 - val_loss: 100.6783
Epoch 5/100
9/9 [==============================] - 0s 6ms/step - loss: 119.0516 - val_loss: 88.7752
Epoch 6/100
9/9 [==============================] - 0s 5ms/step - loss: 112.3047 - val_loss: 81.3722
Epoch 7/100
9/9 [==============================] - 0s 7ms/step - loss: 101.7991 - val_loss: 72.7795
Epoch 8/100
9/9 [==============================] - 0s 8ms/step - loss: 90.6043 - val_loss: 65.9322
Epoch 9/100
9/9 [==============================] - 0s 7ms/step - loss: 82.3838 - val_loss: 59.5307
Epoch 10/100
9/9 [==============================] - 0s 7ms/step - loss: 73.0844 - val_loss: 52.7

## Improving the model by standardizing the dataset

In [79]:
scaler1 = StandardScaler()
X_train_scaler1 = scaler1.fit_transform(X_train)
X_test_scaler1 = scaler1.transform(X_test)

In [84]:
model_reg_1 = tensorflow.keras.models.Sequential([
    tensorflow.keras.layers.Dense(12, activation='leaky_relu', input_shape=(5,)),
    tensorflow.keras.layers.Dense(14, activation='softmax'),
    tensorflow.keras.layers.Dense(14, activation='softmax'),
    tensorflow.keras.layers.Dense(1)
])

model_reg_1.compile(optimizer='adam', loss='mse')

hist_4 = model_reg_1.fit(X_train_scaler1, Y_train, epochs=500, batch_size=32, validation_data=(X_test_scaler1, Y_test))

Epoch 1/500
9/9 [==============================] - 1s 26ms/step - loss: 22.0113 - val_loss: 20.9578
Epoch 2/500
9/9 [==============================] - 0s 6ms/step - loss: 21.8587 - val_loss: 20.8182
Epoch 3/500
9/9 [==============================] - 0s 6ms/step - loss: 21.7065 - val_loss: 20.6789
Epoch 4/500
9/9 [==============================] - 0s 7ms/step - loss: 21.5596 - val_loss: 20.5427
Epoch 5/500
9/9 [==============================] - 0s 8ms/step - loss: 21.4125 - val_loss: 20.4062
Epoch 6/500
9/9 [==============================] - 0s 8ms/step - loss: 21.2672 - val_loss: 20.2747
Epoch 7/500
9/9 [==============================] - 0s 7ms/step - loss: 21.1247 - val_loss: 20.1410
Epoch 8/500
9/9 [==============================] - 0s 8ms/step - loss: 20.9851 - val_loss: 20.0129
Epoch 9/500
9/9 [==============================] - 0s 8ms/step - loss: 20.8426 - val_loss: 19.8772
Epoch 10/500
9/9 [==============================] - 0s 8ms/step - loss: 20.6970 - val_loss: 19.7487
Epoch 11

## Tuning of layers and neurons

### Smaller network

In [85]:
model_small = tensorflow.keras.models.Sequential([
    tensorflow.keras.layers.Dense(3, activation='relu', input_shape=(X_train_scaler1.shape[1],)),
    tensorflow.keras.layers.Dense(3, activation='relu'),
    tensorflow.keras.layers.Dense(1, activation='linear')
])

model_small.compile(optimizer='adam', loss='mse')

# Train the model
hist_S = model_small.fit(X_train_scaler1, Y_train, epochs=100, batch_size=32, validation_data=(X_test_scaler1, Y_test))

print("Smaller model MSE:")
print("Training: ", hist_S.history['loss'][-1])
print("Testing: ", hist_S.history['val_loss'][-1])

Epoch 1/100
9/9 [==============================] - 1s 23ms/step - loss: 24.1143 - val_loss: 23.2144
Epoch 2/100
9/9 [==============================] - 0s 5ms/step - loss: 23.7926 - val_loss: 22.8714
Epoch 3/100
9/9 [==============================] - 0s 7ms/step - loss: 23.4898 - val_loss: 22.5612
Epoch 4/100
9/9 [==============================] - 0s 6ms/step - loss: 23.2145 - val_loss: 22.2703
Epoch 5/100
9/9 [==============================] - 0s 6ms/step - loss: 22.9477 - val_loss: 21.9927
Epoch 6/100
9/9 [==============================] - 0s 6ms/step - loss: 22.6897 - val_loss: 21.7274
Epoch 7/100
9/9 [==============================] - 0s 7ms/step - loss: 22.4411 - val_loss: 21.4627
Epoch 8/100
9/9 [==============================] - 0s 6ms/step - loss: 22.1829 - val_loss: 21.2034
Epoch 9/100
9/9 [==============================] - 0s 7ms/step - loss: 21.9248 - val_loss: 20.9489
Epoch 10/100
9/9 [==============================] - 0s 7ms/step - loss: 21.6666 - val_loss: 20.7095
Epoch 11

### Larger Network

In [86]:
model_large = tensorflow.keras.models.Sequential([
    tensorflow.keras.layers.Dense(36, activation='relu', input_shape=(X_train_scaler1.shape[1],)),
    tensorflow.keras.layers.Dense(36, activation='relu'),
    tensorflow.keras.layers.Dense(36, activation='relu'),
    tensorflow.keras.layers.Dense(20, activation='linear'),
    tensorflow.keras.layers.Dense(1)
])

# Compile the model
model_large.compile(optimizer="adam", loss="mse", metrics=["mse"])

# Train the model
hist_L = model_large.fit(X_train_scaler1, Y_train, epochs=100, batch_size=32, validation_data=(X_test_scaler1, Y_test))

# Print the MSE for the training and testing sets
print("Larger model MSE:")
print("Training: ", hist_L.history['loss'][-1])
print("Testing: ", hist_L.history['val_loss'][-1])

Epoch 1/100
9/9 [==============================] - 2s 28ms/step - loss: 20.7382 - mse: 20.7382 - val_loss: 18.1379 - val_mse: 18.1379
Epoch 2/100
9/9 [==============================] - 0s 8ms/step - loss: 17.1108 - mse: 17.1108 - val_loss: 14.9178 - val_mse: 14.9178
Epoch 3/100
9/9 [==============================] - 0s 8ms/step - loss: 13.6032 - mse: 13.6032 - val_loss: 12.4696 - val_mse: 12.4696
Epoch 4/100
9/9 [==============================] - 0s 6ms/step - loss: 11.2993 - mse: 11.2993 - val_loss: 12.0440 - val_mse: 12.0440
Epoch 5/100
9/9 [==============================] - 0s 7ms/step - loss: 11.2240 - mse: 11.2240 - val_loss: 11.1743 - val_mse: 11.1743
Epoch 6/100
9/9 [==============================] - 0s 6ms/step - loss: 10.7289 - mse: 10.7289 - val_loss: 10.2259 - val_mse: 10.2259
Epoch 7/100
9/9 [==============================] - 0s 6ms/step - loss: 10.2791 - mse: 10.2791 - val_loss: 10.0271 - val_mse: 10.0271
Epoch 8/100
9/9 [==============================] - 0s 6ms/step - los

Dataset used: Liver Disorders. (1990). UCI Machine Learning Repository. https://doi.org/10.24432/C54G67.